# Robust journey planner

- The goal of this notebook running on a local python kernel is to implement a **Robust Journey Planner**, using all of the work that precedes this notebook.

- Given a desired arrival time, the route planner should compute the fastest route between *departure* and *arrival* stops within a provided **confidence tolerance** expressed as interquartiles.

- Notably, our planner should be able to answer the following : "*what route from A to B is the fastest at least Q% of the time if I want to arrive at B before instant T ?".* 

- To that end, we will combine the **graph** used for the *Naive Journey Planner* in the 2nd notebook, with the **Predictive Models of the Delay** built in the 3rd notebook.

- These elements will then be given as inputs to a revisited version of the **Dijkstra's algorithm**, which should assign a confidence score to a path linking a starting point and an end point.

## 1 - Data Loading & Preprocessing

Making the necessary imports, fetching our precomputed data from HDFS, preprocessing it.

In [1]:
# Imports
import os
import functools
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pickle as pkl
import json
import datetime
from ipywidgets import widgets
from IPython.display import display, HTML
from contextlib import contextmanager
from scipy.stats import gamma
from hdfs3 import HDFileSystem

# Configurations
pd.set_option("display.max_columns", 50)
%matplotlib inline

In [2]:
def load_hdfs_to_pandas(filename):
        hdfs = HDFileSystem(host='hdfs://iccluster040.iccluster.epfl.ch', port=8020, user='ebouille') # impersonate ebouille to read the file
        files = hdfs.glob(f'/user/vyuan/final_6/{filename}')
        df = pd.DataFrame()
        for file in files:
            if not 'SUCCESS' in file:
                with hdfs.open(file) as f:
                    df = df.append(pd.read_parquet(f))
        return df

# Load the data
trips = load_hdfs_to_pandas('graph_edges_2.parquet')
graph_nodes = load_hdfs_to_pandas("graph_nodes.parquet")

# Displaying the data
print(f"Length of trips is {len(trips)}")
print(trips.head())
print(f"\nLength of graph_nodes is {len(graph_nodes)}")
graph_nodes.head()

Length of trips is 5692816
                trip_id      src      dst departure_time arrival_time  \
0  1.TA.26-18-j19-1.1.H  8503064  8503065       10:41:00     10:45:00   
1  1.TA.26-18-j19-1.1.H  8503065  8503074       10:45:00     10:46:00   
2  1.TA.26-18-j19-1.1.H  8503074  8503068       10:46:00     10:47:00   
3  1.TA.26-18-j19-1.1.H  8503068  8503066       10:47:00     10:48:00   
4  1.TA.26-18-j19-1.1.H  8503066  8503075       10:48:00     10:50:00   

  duration    type  
0        4  S-Bahn  
1        1  S-Bahn  
2        1  S-Bahn  
3        1  S-Bahn  
4        2  S-Bahn  

Length of graph_nodes is 1950


,id,stop_name,lat,lon
0,8557033,"Oberhasli, Industrie",47.459267,8.490014
1,8573711,"Zürich, Sädlenweg",47.367755,8.487480
2,8591828,"Ebmatingen, Dorf",47.351392,8.641003
3,8590610,"Fällanden, Schützenhaus",47.368625,8.632478
4,8580617,"Bülach, Engelwis",47.511189,8.537130


In [3]:
# Preprocessing the data - changing its columns types, adding key & weight
trips["src"] = trips["src"].astype(str)
trips["dst"] = trips["dst"].astype(str)
trips["type"] = trips["type"].astype(str)
trips["departure_time"] = pd.to_datetime(trips["departure_time"] , format='%H:%M:%S', errors='coerce').dt.time
trips["key"] = trips["arrival_time"].astype(str)
trips["arrival_time"] = pd.to_datetime(trips.arrival_time, format='%H:%M:%S', errors='coerce').dt.time
trips["weight"] = trips["duration"].astype(float)

#### Merging the data with the predictive models of the delays

To match an edge with its given Gamma Distribution to model its potential delays, we must group the edges by the mean of transport and by its time. As the dataset used to build our predictive models, *istdaten*, **solely comprised data regarding Bus & Trains**, the following assumptions will be made :

- *S-Bahn*, *RegioExpress*, *InterRegio*, *Standseilbahn* *Intercity* *Schiff* *Luftseilbahn* *Eurocity* will be considered to have delays that are similarly distributed as the *Zug* in the *istdaten* dataset.
- *Bus*, *Taxi*, *Tram* will be considered to have delays that are similarly distributed as the ones from the *Bus* in the *istdaten* dataset. 
- *Transfers* are not subject to any delays, and therefore do not require any delay modeling.

In [4]:
# Load the gamma distributions modeling the delays for various groups of edges
with open('../data/gamma_distributions.pkl', 'rb') as g_d :
    gamma_distributions = pkl.load(g_d)
    
# One gamma distribution per mean of transport and peak-time/off-peak-time
final_g_d = {}

# Iterate through the parameters
for name, g_p  in gamma_distributions.items():
    m, d, p = name.split(', ')
    agg = (', ').join([m, p])
    if agg in final_g_d:
        f_a = final_g_d[agg]['fit_alpha'] + g_p['fit_alpha']
        f_l = final_g_d[agg]['fit_loc'] + g_p['fit_loc']
        f_s = final_g_d[agg]['fit_scale'] + g_p['fit_scale']
        final_g_d[agg] = {'fit_alpha': f_a, 'fit_loc': f_l, 'fit_scale': f_s}
    else :
        final_g_d[agg] = g_p

# Take the mean over the week days
for name in final_g_d.keys():
    for p_name in final_g_d[name].keys():
        final_g_d[name][p_name] /= 5
    
# Match each edge with its corresponding gamma distribution - Initialization
trips['fit_alpha'] = None
trips['fit_loc'] = None
trips['fit_scale'] = None
transport_mapping = {'Zug': ['S-Bahn', 'RegioExpress', 'InterRegio', 'Standseilbahn', 'Intercity', 'Schiff', 'Luftseilbahn', 'Eurocity'],
                     'Bus': ['Taxi', 'Bus', 'Tram']}
peak_times_bounds = [[datetime.time(7,0,0), datetime.time(8,0,0)], [datetime.time(17,0,0), datetime.time(18,0,0)]]

# Compute the peak time indexes
peak_time_idx = []
for peak_time in peak_times_bounds :
    peak_time_idx.append(((trips.arrival_time<peak_time[1]) & (trips.arrival_time>peak_time[0])) | 
                         ((trips.departure_time<peak_time[1]) & (trips.departure_time>peak_time[0])))
peak_time_idx = peak_time_idx[0] | peak_time_idx[1]

# Case 1 - Zug & Peak-time
case_1_idx = peak_time_idx & trips.type.isin(transport_mapping['Zug'])

# Case 2 - Zug & Off-Peak-time
case_2_idx = -peak_time_idx & trips.type.isin(transport_mapping['Zug'])

# Case 3 - Bus & Peak-time
case_3_idx = peak_time_idx & trips.type.isin(transport_mapping['Bus'])

# Case 4 - Bus & Off-Peak-time
case_4_idx = -peak_time_idx & trips.type.isin(transport_mapping['Bus'])

# Matching
cases = ['Zug, Peak-time', 'Zug, Off-peak-time', 'Bus, Peak-time', 'Bus, Off-peak-time']
for idx, case_idx in enumerate([case_1_idx, case_2_idx, case_3_idx, case_4_idx]) :
    trips.loc[case_idx,'fit_alpha'] = final_g_d[cases[idx]]['fit_alpha']
    trips.loc[case_idx,'fit_loc'] = final_g_d[cases[idx]]['fit_loc']
    trips.loc[case_idx,'fit_scale'] = final_g_d[cases[idx]]['fit_scale']

## 2 - Graph Creation 

We create the graph from the edges using networkx, and add an attribute representing the latest arrival

In [5]:
graph_1 = nx.convert_matrix.from_pandas_edgelist(trips, 
                                                 "src", 
                                                 "dst", 
                                                 edge_key="key",
                                                 edge_attr=["departure_time", "arrival_time", "duration", "type", "weight", 'trip_id', 'fit_alpha', 'fit_loc', 'fit_scale'], 
                                                 create_using=nx.MultiDiGraph())

nx.set_node_attributes(graph_1, datetime.time(0, 0, 0), "latest_arrival")

##### Strongly connected component

As we want our journey planner to always be able to work, i.e. to reach a point B from a point A, we need the different stops to be strongly connected.

Thus, we take the maximum strongly connected component, which includes Zurich train station, to restrict our graph to a strongly connected component.

In [6]:
strongly_connected_components = max(nx.strongly_connected_components(graph_1), key=len)
hbb_in_components = '8503000' in strongly_connected_components
print(f"Our strongly connected component has Zurich Train station {hbb_in_components}")
graph_nodes = graph_nodes[graph_nodes["id"].isin(strongly_connected_components)]
graph_strong_connected = graph_1.subgraph(strongly_connected_components)

Our strongly connected component has Zurich Train station True


## 3 - Robust Journey Planner

Our approach to implement the Robust Journey Planner is the following :

- Store all the incoming edges for a reached node : we must keep all the edges since we will compute a probability of success of each of the path, and make decisions based on this latter. As we are keeping most of the edges, the search grows exponentially in space and time, and thus requires some pruning strategy.

- Pruning strategies : 
    1. Prune any path that changes direction twice (gets away from the destination node twice)
    2. Prune if two consecutive transfers
    3. Prune if the probability of the path is smaller than the confidence threshold
    4. Only keep the *top k* paths that have the k smallest weights as the paths in the graph to be developed.
    
The output of the algorithm should be a list of routes between A and B and their confidence levels. The routes will be sorted from latest (fastest) to earliest (longest) departure time at A, they must all arrive at B before T with a confidence level greater than or equal to Q.

##### Teaser
As a teaser to what follow, let us compute the probability that a mean of transport is subject to a delay that is smaller than 5 minutes, on average, using our results from the 3rd Notebook.

In [7]:
# Computing probabilities
pd.Series([gamma.cdf(300, a = gamma_params['fit_alpha'], loc = gamma_params['fit_loc'], scale = gamma_params['fit_scale']) 
           for _, gamma_params in gamma_distributions.items()]).mean()

0.9785042082486219

Judging from this result, we can safely assume for the most part that we can take any connection whose departure is scheduled 5 minutes after the arrival.

### Routing

All the functions required to run our robust planner are contained in helpers_robust.py.

In [8]:
from helpers_robust import *
routes, probas = final_journey(graph = graph_1.copy(), 
                               nodes = graph_nodes, 
                               start = 'Oberhasli, Industrie', #8557033
                               stop = 'Zürich, Sädlenweg', #8573711
                               time = string_datetime('15:00:00'), 
                               confidence_thres = 0.95, 
                               number_of_routes = 5, 
                               top_k = 30)

To go from Oberhasli, Industrie, to Zürich, Sädlenweg, with a confidence threshold of 0.9, the top 5 routes along with their probabilities can be computed as above. As an example, the first route is shown below.

In [9]:
print('Probability of route = ', probas[0])

Probability of route =  0.999964669437943


In [10]:
routes[0]

,stop,lat,lon,mean_of_transport,departures,arrivals
0,"Oberhasli, Industrie",47.4592670391304,8.49001368400678,None,12:32:00,None
1,"Niederhasli, Bahnhof",47.4782740863515,8.48854044694062,Taxi,13:08:00,12:38:00
2,Niederhasli,47.4783894436049,8.48869316053894,transfer,13:16:00,13:10:00
3,Oberglatt,47.4702652453062,8.51060307032167,S-Bahn,13:20:00,13:20:00
4,Rümlang,47.4540677487502,8.5327465420783,S-Bahn,13:23:00,13:22:00
5,Glattbrugg,47.4310109642626,8.55883361793276,S-Bahn,13:27:00,13:26:00
6,Zürich Oerlikon,47.4118348510255,8.54411023042399,S-Bahn,13:34:00,13:29:00
7,Zürich HB,47.379271132311,8.54019357578468,S-Bahn,13:47:00,13:39:00
8,Zürich Wiedikon,47.3715939887424,8.52345796203921,S-Bahn,14:04:00,13:49:00
9,"Zürich, Schmiede Wiedikon",47.3701521184186,8.51926282966178,transfer,14:13:00,14:13:00


### Testing & Validation of the Robust Planner

Let us check some visualisations for the same route, using our Naive planner against our Robust planner.

In [11]:
from helpers_visualisation import *
from helpers_visualisation_robust import *

In [14]:
plot_path_robust(graph_nodes, graph_1) 